In [12]:
import os

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.extensions.google_cloud_big_query import BigQueryExampleGen
from tfx.v1.components import StatisticsGen, SchemaGen, Transform, Trainer, Evaluator, Pusher
from tfx.types import standard_artifacts
from tfx.proto import pusher_pb2
import tensorflow_model_analysis as tfma

import transform_module  
import trainer_module     

PROJECT_ID = "uci-bank-marketing-ml-lab"
GCP_REGION = "us-central1"

BQ_PROJECT = PROJECT_ID
BQ_DATASET = "bank_data"
BQ_TABLE = "uci_bank_marketing"

PIPELINE_NAME = "bq_classifier_validation"
LOCAL_PIPELINE_ROOT = os.path.join(os.getcwd(), "tfx_local", PIPELINE_NAME)
os.makedirs(LOCAL_PIPELINE_ROOT, exist_ok=True)

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

context = InteractiveContext(pipeline_root=LOCAL_PIPELINE_ROOT)
print("Using pipeline_root:", context.pipeline_root)

TMP_LOCATION = "gs://uci-bank-ml-bucket/tfx/tmp"

beam_pipeline_args = [
    f"--project={PROJECT_ID}",
    f"--region={GCP_REGION}",
    f"--temp_location={TMP_LOCATION}",
]


Using pipeline_root: /home/jupyter/tfx_local/bq_classifier_validation


In [13]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

example_gen = BigQueryExampleGen(query=query)

# This may run quickly and reuse previous BigQuery artifacts
example_gen_result = context.run(example_gen, beam_pipeline_args=beam_pipeline_args)

print("ExampleGen outputs:", example_gen.outputs)


ExampleGen outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=BigQueryExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}


In [14]:


statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(statistics_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 52
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [15]:


schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=True,
)
context.run(schema_gen, beam_pipeline_args=beam_pipeline_args)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 53
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [16]:

transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.join(os.getcwd(), "transform_module.py"),
)

transform_result = context.run(transform, beam_pipeline_args=beam_pipeline_args)

print("Transform outputs:")
print(transform.outputs)
print("  transformed_examples:", transform.outputs["transformed_examples"])
print("  transform_graph:     ", transform.outputs["transform_graph"])

running bdist_wheel
running build
running build_py
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /var/tmp/tmpdzy7nnsj
running install
running install_lib
copying build/lib/transform_module.py -> /var/tmp/tmpdzy7nnsj/.
copying build/lib/trainer_module.py -> /var/tmp/tmpdzy7nnsj/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpdzy7nnsj/./tfx_user_code_Transform-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc875

/home/jupyter/tfx-env/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3-none-any.whl
Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_transform-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3-none-any.whl


Transform outputs:
{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, pr

In [17]:
from tfx.v1.components import Trainer
import os

print("Transformed examples channel:", transform.outputs["transformed_examples"])
print("Transform graph channel:     ", transform.outputs["transform_graph"])

trainer = Trainer(
    module_file=os.path.join(os.getcwd(), "trainer_module.py"),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args={'num_steps': 2000},   # use dict instead of trainer_pb2.TrainArgs
    eval_args={'num_steps': 500},     # use dict instead of trainer_pb2.EvalArgs
)

trainer_result = context.run(trainer, beam_pipeline_args=beam_pipeline_args)

print("Trainer outputs:", trainer.outputs)


Transformed examples channel: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
Transform graph channel:      OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
running bdist_wheel
running build
running build_py
creating build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /var/tmp/tmpkmkrqbav
running install
running install_lib
copying build/lib/transform_module.py -> /var/tmp/tmpkmkrqbav/.
copying build/lib/trainer_module.py -> /var/tmp/tmpkmkrqbav/.
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/d

/home/jupyter/tfx-env/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./tfx_local/bq_classifier_validation/_wheels/tfx_user_code_trainer-0.0+0f8c7c9fcd437087ec2d8b8b16b012c2be2b30424f52706cc97d9564fc8751c8-py3-none-any.whl
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - accuracy: 0.9102 - auc: 0.9320 - loss: 0.1953 - val_accuracy: 0.9124 - val_auc: 0.9376 - val_loss: 0.1866
Trainer outputs: {'model': OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'model_run': OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}


#### Evalconfig 

In [19]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            label_key="y",
            prediction_key="output_0",
            signature_name="serving_default",
        )
    ],
    slicing_specs=[
        tfma.SlicingSpec(),  # Overall metrics
        # You can add more slicings later, e.g. by a feature name:
        # tfma.SlicingSpec(feature_keys=["some_feature"])
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="Accuracy"),
            ]
        )
    ],
)


#### Evaluator component

In [20]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    eval_config=eval_config,
)

evaluator_result = context.run(evaluator, beam_pipeline_args=beam_pipeline_args)

print("Evaluator output artifacts:", evaluator.outputs)


Evaluator output artifacts: {'evaluation': OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False), 'blessing': OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}


In [21]:
serving_model_dir = os.path.join(
    os.getcwd(),
    "serving_model",
    "bq_classifier_validation",
)

print("Serving model will be pushed to:", serving_model_dir)


Serving model will be pushed to: /home/jupyter/serving_model/bq_classifier_validation


#### Pusher component

In [22]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_dir
        )
    ),
)

pusher_result = context.run(pusher, beam_pipeline_args=beam_pipeline_args)

print("Pusher output artifacts:", pusher.outputs)


Pusher output artifacts: {'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}


In [23]:
import glob

pushed_models = glob.glob(os.path.join(serving_model_dir, "*"))
print("Pushed model dirs:", pushed_models)


Pushed model dirs: []
